In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# The first task is answer the following questions using some of the methods we have covered in the lecture/demo.

# How many houses are in this dataset?
# How many features are there for predicting house price?
# Are there any null values in this dataset?
# Which three variables are best correlated with house price (include correlation coefficients)?
# Which three variables are least correlated with house price (include correlation coefficients)?

In [2]:
#Import seattle data
seattlehprices = pd.read_csv('/Users/morganscafidi/Documents/GitHub/geospatial-data-science/labs/lab5/lab5/seattle_house_prices.csv')

In [3]:
# How many houses are in this dataset? 
# There are 19,451 houses in this dataset.
seattlehprices

,price,bedrooms,bathrooms,sqft_living,sqft_lot,yr_built,lat,long
0,538000,3,2.25,2570,7242,1951,47.7210,-122.319
1,180000,2,1.00,770,10000,1933,47.7379,-122.233
2,604000,4,3.00,1960,5000,1965,47.5208,-122.393
3,510000,3,2.00,1680,8080,1987,47.6168,-122.045
4,1230000,4,4.50,5420,101930,2001,47.6561,-122.005
...,...,...,...,...,...,...,...,...
19446,475000,3,2.50,1310,1294,2008,47.5773,-122.409
19447,360000,3,2.50,1530,1131,2009,47.6993,-122.346
19448,400000,4,2.50,2310,5813,2014,47.5107,-122.362
19449,400000,3,2.50,1600,2388,2004,47.5345,-122.069


In [7]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(seattlehprices, geometry=gpd.points_from_xy(seattlehprices['long'], seattlehprices['lat']))
gdf = gdf.set_crs(4326, allow_override=True)

# Reproject everything to UTM 10N (EPSG:32610)
gdf_utm = gdf.to_crs('EPSG:32610')
coast_utm = coast.to_crs('EPSG:32610')

NameError: name 'coast' is not defined

In [8]:
# Compute distance to coast
distance_to_coast = []
for i in range(gdf_utm.shape[0]):
    distance_to_coast.append(coast_utm.distance(gdf_utm['geometry'].iloc[i]).min())
    
# Add to DataFrame
gdf_utm['distance_to_coast'] = distance_to_coast

NameError: name 'coast_utm' is not defined

In [ ]:
# How many features are there for predicting house price?
# There are 8 existing features for predicting house price.

In [10]:
# Are there any null values in this dataset?
# No there are no null values in the dataset
# Check for NaN values and data types
seattlehprices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19451 entries, 0 to 19450
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   price        19451 non-null  int64   
 1   bedrooms     19451 non-null  int64   
 2   bathrooms    19451 non-null  float64 
 3   sqft_living  19451 non-null  int64   
 4   sqft_lot     19451 non-null  int64   
 5   yr_built     19451 non-null  int64   
 6   lat          19451 non-null  float64 
 7   long         19451 non-null  float64 
 8   geometry     19451 non-null  geometry
dtypes: float64(3), geometry(1), int64(5)
memory usage: 1.3 MB


In [11]:
# Compute correlation matrix
corr_matrix = seattlehprices.corr()

# Display just house value correlations
corr_matrix["median_house_value"].sort_values(ascending= False)


KeyError: 'median_house_value'

In [13]:
seattlehprices.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'yr_built',
       'lat', 'long', 'geometry'],
      dtype='object')